In [50]:
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers
import numpy as np
import random
from SoccerWorld import player, game

In [55]:
def friend_action(qtable, player_state, rand_rate, num_actions):
    if np.random.rand() < rand_rate:
        action = np.random.randint(0, num_actions)
    else:
        action = np.argmax(np.max(qtable[player_state, :, :]), axis=0)
        if action > 4:
            print (qtable[player_state, :, :])

    return action



def friend_q_learning():
    
    errors = []
    qs = []
    #hyperparameters
    num_actions = 5
    num_rows = 2
    num_cols = 4
    num_states = num_rows * num_cols * 2 #2 is for possession
#     n = 1000000
    n = 1000000
    gamma = 0.9
    alpha = 0.7
    alpha_dec = 0.99
    rar = 0.5
    radr = 0.99
    rand_rate = rar
    qtableA = np.zeros(shape=(num_states, num_actions, num_actions), dtype='float')
    qtableB = np.zeros(shape=(num_states, num_actions, num_actions), dtype='float')
    verbose = False
    
    i = 0
    errors.append(0)
    qs.append(0)
    while i < n:
        q = 0
        match = game(verbose=verbose, rows=2, columns=4, goalRstart=0, \
                     goalRend=1, aGoal=0, bGoal=3,aPosition=[0,2],\
                     bPosition=[0,1])
        
        initial_state = match.reset()
        
        state, score, done = match.move(4, 4)
        
        
        rand_rate *= radr
        
        if rand_rate < 0.01:
            rand_rate = 0.1
            


        while True:
            q += 1
            old_state = state
            pA_act = friend_action(qtableA, state[0], rand_rate, num_actions)
            pB_act = friend_action(qtableB, state[1], rand_rate, num_actions)

            state, score, done = match.move(pA_act, pB_act)

         
            if q > 100:
                break

            if done:
                qtableA[old_state[0], pA_act, pB_act] = qtableA[old_state[0], pA_act, pB_act] + alpha * (score[0] - qtableA[old_state[0], pA_act, pB_act])
                qtableB[old_state[1], pB_act, pA_act] = qtableB[old_state[1], pB_act, pA_act] + alpha * (score[1] - qtableB[old_state[1], pB_act, pA_act])                   
                break

            else:            
                qtableA[old_state[0], pA_act, pB_act] = qtableA[old_state[0], pA_act, pB_act] + alpha * (score[0] + gamma * (np.max(qtableA[state[0], :, :]) - qtableA[old_state[0], pA_act, pB_act]))          
                qtableB[old_state[1], pB_act, pA_act] = qtableB[old_state[1], pB_act, pA_act] + alpha * (score[1] + gamma * (np.max(qtableB[state[1], :, :]) - qtableB[old_state[1], pB_act, pA_act]))        
                if old_state[0] == 2 and pA_act == 1 and pB_act == 4:
                    
                    error = abs(qtableA[2, 1, 4] - qs[-1])
                    qs.append(qtableA[2,1,4])
                    print (i, qtableA[2, 1, 4], error)
                    errors.append(error)
                    if alpha < 0.001:
                        alpha = 0.001
                    alpha *= alpha_dec
            i += 1
        
        if i % 10000 == 0:
            print (i)
    return errors
        
        

In [56]:
errr = friend_q_learning()

4797 62.999999999999964 62.999999999999964
5979 86.07689999999998 23.076900000000016
7350 94.67389909529999 8.596999095300006
37799 97.92968263578959 3.2557835404896025
44970 99.18258795347096 1.2529053176813676
90653 99.67231890890899 0.48973095543803424
121571 99.86667721174463 0.1943583028356386
145206 99.94496450888671 0.07828729714208293
219919 99.97695824456763 0.03199373568091346
264751 99.99021914039749 0.01326089582985901
269753 99.99579188988218 0.005572749484699102
331452 99.99816552952115 0.002373639638960867
458390 99.99918993905558 0.0010244095344376092
480572 99.99963777172603 0.00044783267044579134
492559 99.99983602283268 0.00019825110665294687
526399 99.9999248716903 8.88488576151758e-05
540271 99.99996517189767 4.030020737388895e-05
556307 99.9999836675081 1.8495610433433285e-05
557416 99.99999225421233 8.586704225876929e-06
558649 99.99999628578817 4.031575841167978e-06
576075 99.99999819965234 1.913864167590873e-06
605340 99.9999991180611 9.184087588209877e-07
6068

In [47]:
errr

[0,
 6.220743405298769,
 7.358196382242202,
 7.62296916189473,
 6.881193805344235,
 6.217477609080383,
 5.6231544922356065,
 5.090454997482311,
 4.612533143766491,
 4.183350668563321,
 3.797576492957255,
 3.450499318244198,
 3.137951561829162,
 2.8562430948783657,
 2.6021034612999614,
 2.3726314429113273,
 2.165250993935757,
 1.9776727033322317,
 1.807860059337557,
 1.65399988989779,
 1.51447643784131,
 1.3878486027824835,
 1.2728299446056184,
 1.168271097460618,
 1.0731442897758683,
 0.9865297059412939,
 0.9076034599552258,
 0.8356269812453831,
 0.7699376387371473,
 0.709940451618877,
 0.6551007546348728,
 0.6049377025355938,
 0.5590185128904324,
 0.5169533591261484,
 0.47839083665522253,
 0.443013934529219,
 0.41053645338516276,
 0.38069981771474204,
 0.3532702368193412,
 0.328036174343211,
 0.30480609110661305,
 0.2834064301844137,
 0.26367981687162967,
 0.24548344941389644,
 0.22868765921856493,
 0.2131746217503121,
 0.19883720150087925,
 0.1855779163417992,
 0.1733080082570666,
 0